In [7]:
!pip install webdriver_manager

In [11]:
import time
import os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import datetime
import re
import openpyxl

def main(urls):

    # ドライバ準備
    chrome_options = webdriver.ChromeOptions()
    #Program Files (x86)配下のバージョンの低いChromeを参照しに行ってバージョン非対応と怒られたので、新しいバージョンのChromeアプリにPathを通す
    chrome_options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe" 
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    all_reviews = []
    try:
        for url in urls:
            
            # アクセス
            print("スクレイピング開始: ", url)
            driver.get("https://review.kakaku.com/review/" + url + "#tab")
            
            # モデル取得
            try:
                model_name = driver.find_element(By.CLASS_NAME, 'p-main_title').find_element(By.TAG_NAME, 'h2').text
            except NoSuchElementException:
                model_name = ""
            print("モデル: ", model_name)
            
            # レビュー抽出
            all_reviews.extend(get_review(driver, model_name))

            # ページング処理
            while True:
                try:
                    # ボタンが存在するかチェック
                    load_more_button = driver.find_element(By.CLASS_NAME, 'arrowNext01')
                    load_more_button.click()
                    
                    # 読み込み待機
                    time.sleep(2)
                    
                    # 取得
                    all_reviews.extend(get_review(driver, model_name))

                except NoSuchElementException:
                    print("スクレイピング完了")
                    break
                except TimeoutException:
                    print("タイムアウト発生")
                    break

    finally:
        driver.quit()
    
    # 不正な文字を削除
    df = pd.DataFrame(all_reviews)
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = df[column].apply(remove_illegal_characters)

    # 保存先ディレクトリの指定と作成
    output_dir = os.path.join(os.getcwd(), '..', 'Output') #1個上の階層にあるOutputフォルダに格納
    os.makedirs(output_dir, exist_ok=True)
    
    # ファイル名生成
    now = datetime.datetime.now()
    file_name = os.path.join(output_dir, 'kakakucom_reviews_{}.xlsx'.format(now.strftime('%Y%m%d_%H%M%S')))

    # 保存    
    df.to_excel(file_name, index=False)
    print("Saved to", file_name)   
    

def get_review(driver, model_name):
    reviews = []
    
    review_elements = driver.find_elements(By.CLASS_NAME, 'reviewBox')
    for elem in review_elements:
        
        # パンくずリスト
        try:
            breadcrumbs = elem.find_element(By.CLASS_NAME, 'breadcrumbs').text
        except NoSuchElementException:
            breadcrumbs = ""
                
        # パンくずリストの最終項目からSKUを取得
        try:
            breadcrumbs_elements = elem.find_elements(By.XPATH, './/p[@class="breadcrumbs"]//a')
            if breadcrumbs_elements:
                breadcrumbs_last = breadcrumbs_elements[-1].text
            else:
                breadcrumbs_last = elem.find_element(By.XPATH, './/p[@class="breadcrumbs"]').text.split(' > ')[-1]
        except NoSuchElementException:
            breadcrumbs_last = ""
        
        # ユーザ名
        try:
            user_name = elem.find_element(By.CLASS_NAME, 'userName').text
        except NoSuchElementException:
            user_name = ""
        
        # レビュー投稿日
        try:
            entry_date = elem.find_element(By.CLASS_NAME, 'entryDate').text
        except NoSuchElementException:
            entry_date = ""
        
        # レビュー本文
        try:
            review_content = elem.find_element(By.CLASS_NAME, 'revEntryCont').text
        except NoSuchElementException:
            review_content = ""
            
        # レーティング
        ratings = {}
        try:
            rating_elements = elem.find_elements(By.XPATH, './/div[@class="revRateBox type2"]//table//tr')
            for rating in rating_elements:
                rating_category = rating.find_element(By.TAG_NAME, 'th').text
                rating_value = rating.find_element(By.TAG_NAME, 'td').text
                ratings[rating_category] = rating_value                
        except NoSuchElementException:
            print("error")
            
        reviews.append({
            "model_name": model_name,
            "model_detail": breadcrumbs_last,
            "breadcrumbs": breadcrumbs,
            "url": driver.current_url,
            "user_name": user_name,
            "entry_date": entry_date,
            "review_content": review_content,
            **ratings
        })
    
    return reviews

# 処理分割用
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# 不正文字出力回避用
def remove_illegal_characters(text):
    ILLEGAL_CHARACTERS_RE = re.compile(
        r'[\000-\010]|[\013-\014]|[\016-\037]|[\x00-\x1f\x7f-\x9f]|[\uffff]'
    )
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub("", text)
    return text

url_list = pd.read_excel('../Output/kakakucom_device_lists_20240602_010957.xlsx', index_col=0)
urls = url_list['model_id'].values.tolist()

url_chunks = chunk_list(urls, 10)
for i, url_chunk in enumerate(url_chunks):
    
    # 一時的処理
#     if i < 7:
#         continue
    
    print(f'{i}回目のファイル出力処理開始')
    main(url_chunk)
    
# urls = [
#     # WRITE ME
#     "M0000001024"
# ]
# main(urls)


0回目のファイル出力処理開始
スクレイピング開始:  M0000001058
モデル:  Google Pixel 8a
スクレイピング完了
スクレイピング開始:  M0000001034
モデル:  AQUOS sense8
スクレイピング完了
スクレイピング開始:  M0000001005
モデル:  Google Pixel 7a
スクレイピング完了
スクレイピング開始:  M0000001063
モデル:  Redmi Note 13 Pro 5G
スクレイピング完了
スクレイピング開始:  M0000001024
モデル:  iPhone 15
スクレイピング完了
スクレイピング開始:  M0000001070
モデル:  
スクレイピング完了
スクレイピング開始:  M0000001064
モデル:  Redmi Note 13 Pro+ 5G
スクレイピング完了
スクレイピング開始:  M0000001075
モデル:  POCO F6 Pro
スクレイピング完了
スクレイピング開始:  M0000001055
モデル:  Galaxy S24
スクレイピング完了
スクレイピング開始:  M0000001011
モデル:  OPPO Reno9 A
スクレイピング完了
Saved to C:\Users\tk5de\sophia_datascience_root\Python\thesis\github\Jupyter\..\Output\kakakucom_reviews_20240602_163113.xlsx
1回目のファイル出力処理開始
スクレイピング開始:  M0000000934
モデル:  iPhone SE (第3世代)
スクレイピング完了
スクレイピング開始:  M0000001062
モデル:  Xiaomi 14 Ultra
スクレイピング完了
スクレイピング開始:  M0000001004
モデル:  AQUOS wish3
スクレイピング完了
スクレイピング開始:  M0000001000
モデル:  Xperia 1 V
スクレイピング完了
スクレイピング開始:  M0000001054
モデル:  Galaxy S24 Ultra
スクレイピング完了
スクレイピング開始:  M0000001035
モデル:  Google

モデル:  Xperia XZ1
スクレイピング完了
スクレイピング開始:  M0000000773
モデル:  Galaxy A41
スクレイピング完了
スクレイピング開始:  M0000000839
モデル:  OPPO A73
スクレイピング完了
スクレイピング開始:  M0000000957
モデル:  POCO F4 GT
スクレイピング完了
スクレイピング開始:  M0000000945
モデル:  AQUOS sense6s
スクレイピング完了
スクレイピング開始:  M0000000880
モデル:  AQUOS R6
スクレイピング完了
スクレイピング開始:  M0000000990
モデル:  かんたんスマホ3
スクレイピング完了
スクレイピング開始:  M0000000553
モデル:  iPhone 8
スクレイピング完了
スクレイピング開始:  M0000000630
モデル:  iPhone XR
スクレイピング完了
Saved to C:\Users\tk5de\sophia_datascience_root\Python\thesis\github\Jupyter\..\Output\kakakucom_reviews_20240602_193825.xlsx
13回目のファイル出力処理開始
スクレイピング開始:  M0000000965
モデル:  iPhone 14 Plus
スクレイピング完了
スクレイピング開始:  M0000000914
モデル:  らくらくスマートフォン F-52B
スクレイピング完了
スクレイピング開始:  M0000000904
モデル:  iPhone 13 Pro Max
スクレイピング完了
スクレイピング開始:  M0000000857
モデル:  TORQUE 5G
スクレイピング完了
スクレイピング開始:  M0000000919
モデル:  Xiaomi 11T
スクレイピング完了
スクレイピング開始:  M0000000737
モデル:  OPPO A5 2020
スクレイピング完了
スクレイピング開始:  M0000000606
モデル:  Galaxy S9
スクレイピング完了
スクレイピング開始:  M0000000869
モデル:  Galaxy S21 Ultra 5G
スクレイ

スクレイピング完了
スクレイピング開始:  M0000000590
モデル:  BASIO3
スクレイピング完了
スクレイピング開始:  M0000000527
モデル:  Xperia XZs
スクレイピング完了
スクレイピング開始:  M0000000641
モデル:  AQUOS zero
スクレイピング完了
スクレイピング開始:  M0000000535
モデル:  Galaxy S8+
スクレイピング完了
スクレイピング開始:  M0000000649
モデル:  ROG Phone
スクレイピング完了
スクレイピング開始:  M0000000841
モデル:  AQUOS sense4 basic
スクレイピング完了
スクレイピング開始:  M0000000108
モデル:  iPhone 4S
スクレイピング完了
Saved to C:\Users\tk5de\sophia_datascience_root\Python\thesis\github\Jupyter\..\Output\kakakucom_reviews_20240602_221340.xlsx
25回目のファイル出力処理開始
スクレイピング開始:  M0000000490
モデル:  Xperia X Compact
スクレイピング完了
スクレイピング開始:  M0000000800
モデル:  かんたんスマホ2
スクレイピング完了
スクレイピング開始:  M0000000632
モデル:  iPhone XS Max
スクレイピング完了
スクレイピング開始:  M0000000645
モデル:  らくらくスマートフォン me F-01L
スクレイピング完了
スクレイピング開始:  M0000000296
モデル:  iPhone 6
スクレイピング完了
スクレイピング開始:  M0000000756
モデル:  Galaxy Z Flip
スクレイピング完了
スクレイピング開始:  M0000000268
モデル:  Xperia Z Ultra
スクレイピング完了
スクレイピング開始:  M0000000438
モデル:  Xperia X Performance
スクレイピング完了
スクレイピング開始:  M0000000250
モデル:  Xperia Z1
スクレイピング完了

モデル:  BASIO
スクレイピング完了
スクレイピング開始:  M0000000342
モデル:  Galaxy S6
スクレイピング完了
スクレイピング開始:  M0000000508
モデル:  Android One S1
スクレイピング完了
スクレイピング開始:  M0000000753
モデル:  ZenFone Max Pro (M2) (RAM 6GBモデル)
スクレイピング完了
スクレイピング開始:  M0000000389
モデル:  arrows M02
スクレイピング完了
スクレイピング開始:  M0000000765
モデル:  HUAWEI Mate 30 Pro 5G
スクレイピング完了
スクレイピング開始:  M0000000262
モデル:  Nexus 5
スクレイピング完了
Saved to C:\Users\tk5de\sophia_datascience_root\Python\thesis\github\Jupyter\..\Output\kakakucom_reviews_20240603_004607.xlsx
37回目のファイル出力処理開始
スクレイピング開始:  M0000000824
モデル:  AQUOS zero5G basic
スクレイピング完了
スクレイピング開始:  M0000000364
モデル:  HUAWEI P8 lite
スクレイピング完了
スクレイピング開始:  M0000000503
モデル:  ZenFone 3 Max
スクレイピング完了
スクレイピング開始:  M0000000513
モデル:  HUAWEI nova
スクレイピング完了
スクレイピング開始:  M0000000894
モデル:  Zenfone 8 (RAM 16GBモデル)
スクレイピング完了
スクレイピング開始:  M0000000301
モデル:  AQUOS ZETA SH-01G
スクレイピング完了
スクレイピング開始:  M0000000297
モデル:  iPhone 6 Plus
スクレイピング完了
スクレイピング開始:  M0000000705
モデル:  arrows RX
スクレイピング完了
スクレイピング開始:  M0000000548
モデル:  arrows M04
スクレイピング完了

スクレイピング完了
スクレイピング開始:  M0000000579
モデル:  HUAWEI Mate 10 lite
スクレイピング完了
スクレイピング開始:  M0000000029
モデル:  SC-01B
スクレイピング完了
スクレイピング開始:  M0000000230
モデル:  らくらくスマートフォン 2
スクレイピング完了
スクレイピング開始:  M0000000784
モデル:  HUAWEI P40 lite E
スクレイピング完了
スクレイピング開始:  M0000000146
モデル:  AQUOS PHONE SERIE
スクレイピング完了
スクレイピング開始:  M0000000254
モデル:  GALAXY J
スクレイピング完了
スクレイピング開始:  M0000000328
モデル:  DIGNO U
スクレイピング完了
Saved to C:\Users\tk5de\sophia_datascience_root\Python\thesis\github\Jupyter\..\Output\kakakucom_reviews_20240603_023346.xlsx
49回目のファイル出力処理開始
スクレイピング開始:  M0000000473
モデル:  ZenFone 3 Deluxe
スクレイピング完了
スクレイピング開始:  M0000000315
モデル:  TORQUE SKT01
スクレイピング完了
スクレイピング開始:  M0000000350
モデル:  AQUOS EVER SH-04G
スクレイピング完了
スクレイピング開始:  M0000000496
モデル:  AQUOS U SHV37
スクレイピング完了
スクレイピング開始:  M0000000124
モデル:  GALAXY NEXUS
スクレイピング完了
スクレイピング開始:  M0000000359
モデル:  URBANO V02
スクレイピング完了
スクレイピング開始:  M0000000418
モデル:  AQUOS SERIE mini SHV33
スクレイピング完了
スクレイピング開始:  M0000000494
モデル:  AQUOS SH-M04
スクレイピング完了
スクレイピング開始:  M0000000539
モデル:  Qu

60回目のファイル出力処理開始
スクレイピング開始:  M0000000001
モデル:  W-ZERO3
スクレイピング完了
スクレイピング開始:  M0000000094
モデル:  AQUOS PHONE IS13SH
スクレイピング完了
スクレイピング開始:  M0000000155
モデル:  AQUOS PHONE sv
スクレイピング完了
スクレイピング開始:  M0000000261
モデル:  Disney Mobile F-03F
スクレイピング完了
スクレイピング開始:  M0000000289
モデル:  STREAM S
スクレイピング完了
スクレイピング開始:  M0000000325
モデル:  miraie
スクレイピング完了
スクレイピング開始:  M0000000406
モデル:  DIGNO rafre KYV36
スクレイピング完了
スクレイピング開始:  M0000000479
モデル:  STAR WARS mobile
スクレイピング完了
スクレイピング開始:  M0000000145
モデル:  AQUOS PHONE CL
スクレイピング完了
スクレイピング開始:  M0000000168
モデル:  PANTONE 5
スクレイピング完了
Saved to C:\Users\tk5de\sophia_datascience_root\Python\thesis\github\Jupyter\..\Output\kakakucom_reviews_20240603_033450.xlsx
61回目のファイル出力処理開始
スクレイピング開始:  M0000000170
モデル:  ARROWS A 101F
スクレイピング完了
スクレイピング開始:  M0000000183
モデル:  HONEY BEE 201K
スクレイピング完了
スクレイピング開始:  M0000000203
モデル:  スマートフォン for ジュニア
スクレイピング完了
スクレイピング開始:  M0000000239
モデル:  あんしんファミリーケータイ
スクレイピング完了
スクレイピング開始:  M0000000360
モデル:  AQUOS CRYSTAL 2
スクレイピング完了
スクレイピング開始:  M0000000495
モデル: